## Import Utils

In [1]:
!pip install keras

In [39]:
import keras
import h5py as h5
import numpy as np

PATH_DATA = 'data/train/eightieth.h5'
PATH_PREDICT_WITHOUT_GT = 'data/pred_students/pred_eighties_from_half_1_without_gt.h5'
PATH_SUBMIT = 'hitogram_try.h5'
PATH_PREDICT_WITH_GT = 'data/pred_teachers/pred_eighties_from_half_1.h5'

In [3]:
BATCH_SIZE = 32
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D, Reshape
import keras.layers.normalization 
from keras.callbacks import Callback
%load_ext autoreload
%autoreload 2

In [4]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion):
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [5]:
idxs = get_idxs(PATH_DATA)
shuffled_idxs = shuffle_idx(idxs)
train_idxs, val_idxs = split_train_val(shuffled_idxs, 0.2)

In [21]:
def generator_histogram(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
            batch_idxs = sorted(batch_idxs)
            X = []
            Y = f['TOP_LANDCOVER'][batch_idxs, :]
            for bb in batch_idxs:
                patch = f['S2'][bb, :,:,:].flatten()
                h0, bins = np.histogram(f['S2'][bb, :,:,0].flatten(), bins=range(9000))
                h1, bins = np.histogram(f['S2'][bb, :,:,1].flatten(), bins=range(9000))
                h2, bins = np.histogram(f['S2'][bb, :,:,2].flatten(), bins=range(9000))
                h3, bins = np.histogram(f['S2'][bb, :,:,3].flatten(), bins=range(9000))
                h = np.concatenate((patch, h0, h1, h2, h3), axis=0)
                X.append(h)
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 23)

In [22]:
train_gen = generator_histogram(PATH_DATA, BATCH_SIZE, train_idxs)
train_batch_count = get_batch_count(train_idxs, BATCH_SIZE)

val_gen = generator_histogram(PATH_DATA, BATCH_SIZE, val_idxs)
val_batch_count = get_batch_count(val_idxs, BATCH_SIZE)
print(train_batch_count, val_batch_count)
print(np.shape(train_gen.__next__()[0])) 


5850 1463
(32, 37020)


# Instanciation du model

In [26]:
input_shape = (37020,)
model_h = Sequential()
model_h.add(BatchNormalization(input_shape=input_shape))
model_h.add(Activation('relu'))
model_h.add(Dense(23))
model_h.add(Activation('softmax'))

In [27]:
optim = keras.optimizers.Adam(lr=0.001)

model_h.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_h.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 37020)             148080    
_________________________________________________________________
activation_7 (Activation)    (None, 37020)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 23)                851483    
_________________________________________________________________
activation_8 (Activation)    (None, 23)                0         
Total params: 999,563
Trainable params: 925,523
Non-trainable params: 74,040
_________________________________________________________________


In [32]:
history_h = model_h.fit_generator(train_gen, steps_per_epoch=train_batch_count/BATCH_SIZE, epochs=2, verbose=1, 
                              validation_data=val_gen, validation_steps=val_batch_count/BATCH_SIZE)
model_h.save_weights('hitogram_try.h5')

Epoch 1/2
183/182 [==============================] - 183s 999ms/step - loss: 1.6951 - acc: 0.4986 - val_loss: 1.8585 - val_acc: 0.4769
Epoch 2/2
183/182 [==============================] - 177s 966ms/step - loss: 1.7010 - acc: 0.4822 - val_loss: 1.8431 - val_acc: 0.5007


In [33]:
score_h = model_h.evaluate_generator(val_gen, steps=val_batch_count/BATCH_SIZE)
print('Test loss:', score_h[0])
print('Test accuracy:', score_h[1])

Test loss: 1.9320692186770232
Test accuracy: 0.5129076086956522


## Prediction routines

In order to submit a result here are some gits

In [ ]:
import os 
def prediction_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        X = []
        for bb in batch_idxs:
                patch = f['S2'][bb, :,:,:].flatten()
                h0, bins = np.histogram(f['S2'][bb, :,:,0].flatten(), bins=range(9000))
                h1, bins = np.histogram(f['S2'][bb, :,:,1].flatten(), bins=range(9000))
                h2, bins = np.histogram(f['S2'][bb, :,:,2].flatten(), bins=range(9000))
                h3, bins = np.histogram(f['S2'][bb, :,:,3].flatten(), bins=range(9000))
                h = np.concatenate((patch, h0, h1, h2, h3), axis=0)
                X.append(h)
        yield np.array(X)

def build_h5_pred_file(pred, h5_output_path):
    if os.path.exists(h5_output_path):
        os.remove(h5_output_path)
    f = h5.File(h5_output_path, 'w')
    top_landcover_submit = f.create_dataset("TOP_LANDCOVER", (len(pred), 1), maxshape=(None, 1))
    top_landcover_submit[:, 0] = pred
    f.close()
    
    return 1

In [ ]:
pred_idx = get_idxs(PATH_PREDICT_WITHOUT_GT)
print(len(pred_idx))
pred_gen = prediction_generator(PATH_PREDICT_WITHOUT_GT, BATCH_SIZE, pred_idx)
prediction = model_h.predict_generator(pred_gen, steps=get_batch_count(pred_idx, BATCH_SIZE), verbose=1)
print(len(prediction))
build_h5_pred_file(np.argmax(prediction, axis = 1), PATH_SUBMIT)

241700
7513/7554 [============================>.] - ETA: 30s

## Some ideas for monitoring

In [42]:
def gt_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        Y = f['TOP_LANDCOVER'][batch_idxs, :]
        yield keras.utils.np_utils.to_categorical(np.array(Y), 23)

gt_gen = gt_generator(PATH_PREDICT_WITH_GT, BATCH_SIZE, pred_idx)
gt = []
for elem in gt_gen:
    gt.append(elem)
gt = np.vstack(gt)

In [44]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",fontsize=7)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [45]:
def clean_confusion_matrix(confusion_matrix, classes):
    real_classes = []
    for c in range(len(classes)):
        if np.sum(confusion_matrix[:,c])+np.sum(confusion_matrix[c, :]) != 0:
            real_classes.append(c)
    real_confusion_matrix = np.empty((len(real_classes), len(real_classes)))  
    for c_index in range(len(real_classes)):
        real_confusion_matrix[c_index,:] = confusion_matrix[real_classes[c_index], real_classes]
    return real_confusion_matrix, real_classes

In [46]:
%matplotlib notebook
from sklearn.metrics import confusion_matrix
y_true = np.argmax(gt, axis=1)
y_pred = np.argmax(prediction, axis = 1)

real_cnf_matrix, real_classes = clean_confusion_matrix(confusion_matrix(y_true, y_pred, labels= range(23)), range(23))
plot_confusion_matrix(real_cnf_matrix, classes = real_classes, normalize=True)

ValueError: Found input variables with inconsistent numbers of samples: [241700, 7584]